In [2]:
import os
from langchain.chat_models import init_chat_model
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

model = init_chat_model("groq:llama-3.1-8b-instant")

model.invoke("Hello, how are you?").content


"I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have. How can I assist you today?"

In [ ]:
###Tools

from langchain.tools import tool


@tool
def get_weather(location: str) -> str:
    """Get the weather for a location."""
    return f"The sunny in {location} ."


@tool
def fetch_temperature(location: str) -> str:
    """Get the temperature for a location."""
    return f"The temperature in {location} is 30 Celsius."


model_with_tools = model.bind_tools([get_weather, fetch_temperature])


In [34]:
response = model_with_tools.invoke("What is the weather and temperature in Bangalore?")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'xrwsbyh99', 'function': {'arguments': '{"location":"Bangalore"}', 'name': 'fetch_temperature'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 221, 'total_tokens': 237, 'completion_time': 0.020390878, 'completion_tokens_details': None, 'prompt_time': 0.03339618, 'prompt_tokens_details': None, 'queue_time': 0.05989469, 'total_time': 0.053787058}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6bc9-edaa-7092-ac6e-65ea0c329a94-0', tool_calls=[{'name': 'fetch_temperature', 'args': {'location': 'Bangalore'}, 'id': 'xrwsbyh99', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 221, 'output_tokens': 16, 'total_tokens': 237})

In [35]:
response.tool_calls

[{'name': 'fetch_temperature',
  'args': {'location': 'Bangalore'},
  'id': 'xrwsbyh99',
  'type': 'tool_call'}]

## Tool Execution Loop

In [19]:
#Step 1: Model generates tool calls
message = [{"role":"user","content":"What is the weather in Bangalore?"}]
ai_msg = model_with_tools.invoke(message)
message.append(ai_msg)

#Step 2: Tool calls are executed
for tool_call in ai_msg.tool_calls:
    #Execute tool call
    tool_result = get_weather.invoke(tool_call)
    #Add tool result to message
    message.append(tool_result)

#Step 3: Model generates final response
final_response = model_with_tools.invoke(message)
print(final_response.content)


However, since the function 'get_weather' returns more information than just the weather description, let's try to get the current weather in Bangalore.




In [36]:
message

[{'role': 'user', 'content': 'What is the weather in Bangalore?'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qjdzz1ph2', 'function': {'arguments': '{"location":"Bangalore"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 219, 'total_tokens': 234, 'completion_time': 0.061489892, 'completion_tokens_details': None, 'prompt_time': 0.219383437, 'prompt_tokens_details': None, 'queue_time': 0.188347691, 'total_time': 0.280873329}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6bc2-efc0-7722-b550-610cc43f2c4f-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bangalore'}, 'id': 'qjdzz1ph2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 219, 'output_tokens': 15, 'total_tokens': 234}),
 ToolMessage(conten

In [37]:
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x10de73610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10dff0050>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'fetch_temperature', 'description': 'Get the temperature for a location.', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])